# Corona Impact Analysis

TODO: Explanation and links to code here

In [4]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2
from functools import partial
from datetime import date

import ipywidgets as widgets
import numpy as np
import pandas as pd
from pandas.testing import assert_series_equal
import seaborn as sns

from seir import Seir

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
sns.set_style("white")

In [6]:
social_distancing_steps = [
    ("2020-02-28", 0.0),
    ("2020-03-21", 0.85),
    ("2020-04-14", 0.0),
    ("2020-12-01", 0.0),
]

parameters = {
    "beta": 0.5,
    "t_incubation": 5.5,
    "t_presymptomatic": 1.5,
    "t_recovery_asymptomatic": 5,
    "t_recovery_mild": 14,
    "t_recovery_severe": 30.5,
    "t_death": 14,
    "t_hospital_lag": 5,
    "p_asymptomatic": 0.3,
    "p_severe": 0.18,
    "p_fatal": 0.02,
    "p_self_quarantine": 0.5,
    "p_icu_given_hospital": 0.20,
}

start = {
    "T": 0.0,
    "S": (1 - 1 / 10000),
    "E": 1 / 10000,
    "I": 0.0,
    "I_asymptomatic": 0.0,
    "I_mild": 0.0,
    "I_severe_home": 0.0,
    "I_severe_hospital": 0.0,
    "I_fatal_home": 0.0,
    "I_fatal_hospital": 0.0,
    "R_from_asymptomatic": 0.0,
    "R_from_mild": 0.0,
    "R_from_severe": 0.0,
    "Dead": 0,
}

In [8]:
def update_results(
    start,
    params,
    # Policy
    policy_period0,
    policy_period1,
    policy_period2,
    policy_period3,
    date_period0,
    date_period1,
    date_period2,
    date_period3,
    # Parameters
    # Starting Values
):

    policy = [
        (date_period0, policy_period0),
        (date_period1, policy_period1),
        (date_period2, policy_period2),
        (date_period3, policy_period3),
    ]

    # params={}
    # start={}

    # Run
    seir = Seir(params=params, start=start)
    seir.simulate(policy)
    seir.plot_summary()
    return None


update_results_partial = partial(update_results, start=start, params=parameters)

In [9]:
def policy_widget(description, date, policy=0):
    text_box = widgets.Text(value=date, description=description, disabled=False,)
    float_box = widgets.FloatSlider(
        value=policy,
        min=0,
        max=1,
        step=0.1,
        description="Policy Strength",
        disabled=False,
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format=".1f",
    )
    return text_box, float_box


# Policy Box
text_box0, float_box0 = policy_widget("Initial Period", "2020-02-28")
text_box1, float_box1 = policy_widget("Lockdown", "2020-03-21", policy=0.85)
text_box2, float_box2 = policy_widget("Relaxation", "2020-04-14")
text_box3, float_box3 = policy_widget("Normalization", "2020-12-01")

policy_box = widgets.VBox(
    [
        widgets.HBox([text_box0, float_box0]),
        widgets.HBox([text_box1, float_box1]),
        widgets.HBox([text_box2, float_box2]),
        widgets.HBox([text_box3, float_box3]),
    ]
)

# Parameters (placeholder)
parameters_box = widgets.VBox()

# Starting Values (placeholder)
start_box = widgets.VBox()

# Combine inputs in tabs
input_tab = widgets.Tab()
input_tab.children = [policy_box, parameters_box, start_box]
input_tab.set_title(0, "Policy")
input_tab.set_title(1, "Parameters")
input_tab.set_title(2, "Starting Values")

# Output box
output_box = widgets.interactive_output(
    update_results_partial,
    {
        "policy_period0": float_box0,
        "policy_period1": float_box1,
        "policy_period2": float_box2,
        "policy_period3": float_box3,
        "date_period0": text_box0,
        "date_period1": text_box1,
        "date_period2": text_box2,
        "date_period3": text_box3,
    },
)

# Combination
dash = widgets.VBox([output_box, input_tab])
dash